# Imports & Constants

In [1]:
# TODO: this needs to be downloaded from GITHUB artifacts
# from google.colab import drive
# drive.mount('/content/drive/')
# % cp '/content/drive/MyDrive/audiovisual_fusion_tutorial/audiovisual_data.zip' '/content/'


Mounted at /content/drive/


In [ ]:
! unzip 'audiovisual_data.zip'


In [ ]:
% pip install bert-for-tf2

In [4]:
'''
Notebook for the worksheet 4 of the HCIR Lecture
'''

# System imports
import time
import sys
sys.path.append('/content/')

# 3rd party imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator 
import tensorflow_hub as hub
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
# local imports
from utils import FusedData, TextData
# end file header
__author__      = 'Adrian Lubitz'

img_width = img_height = 299
num_test_samples = 22716
num_classes = 101



# Calculate Accuracy for Image Model

## Load Model

In [5]:
image_model = keras.models.load_model('inception_model_weights_0.72.hdf5', custom_objects={'KerasLayer': hub.KerasLayer})

## Load Data

In [6]:
batch_size = 500 # HINT: choose smaller batch_size if your machine runs out of memory
test_datagen = ImageDataGenerator()
validation_generator = test_datagen.flow_from_directory(
    'images/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')
label_map = (validation_generator.class_indices) # this may be helpful for accuracy per class

Found 22716 images belonging to 101 classes.


## Calculate Accuracy

In [7]:
correct_classifications = 0
i = 0
for images, true_labels in validation_generator:
      pred_labels = image_model.predict(images)
      batch_correct = 0
      for actual_label, label in zip(true_labels, pred_labels):
            if actual_label.argmax(0) == label.argmax(0): 
                  correct_classifications += 1
                  batch_correct += 1
      batch_accuracy = batch_correct / len(pred_labels)
      i += 1
      print(f'accuracy for batch {i}/{num_test_samples / batch_size} = {batch_accuracy}')
      if i > num_test_samples / batch_size:
        break
      

accuracy = float(correct_classifications) / num_test_samples
print(f'accuracy = {accuracy}')


accuracy for batch 1/45.432 = 0.7
accuracy for batch 2/45.432 = 0.746
accuracy for batch 3/45.432 = 0.736
accuracy for batch 4/45.432 = 0.744
accuracy for batch 5/45.432 = 0.714
accuracy for batch 6/45.432 = 0.726
accuracy for batch 7/45.432 = 0.722
accuracy for batch 8/45.432 = 0.732
accuracy for batch 9/45.432 = 0.702
accuracy for batch 10/45.432 = 0.718
accuracy for batch 11/45.432 = 0.69
accuracy for batch 12/45.432 = 0.734
accuracy for batch 13/45.432 = 0.728
accuracy for batch 14/45.432 = 0.734
accuracy for batch 15/45.432 = 0.752
accuracy for batch 16/45.432 = 0.714
accuracy for batch 17/45.432 = 0.738
accuracy for batch 18/45.432 = 0.724
accuracy for batch 19/45.432 = 0.682
accuracy for batch 20/45.432 = 0.706
accuracy for batch 21/45.432 = 0.704
accuracy for batch 22/45.432 = 0.694
accuracy for batch 23/45.432 = 0.69
accuracy for batch 24/45.432 = 0.702
accuracy for batch 25/45.432 = 0.74
accuracy for batch 26/45.432 = 0.72
accuracy for batch 27/45.432 = 0.712
accuracy for bat

# Calculate Accuracy for Text Model

## Load Model

In [ ]:
text_model = keras.models.load_model('BERT_LSTM_weights_0.84.hdf5', custom_objects={'KerasLayer': hub.KerasLayer})

## Load Data

In [ ]:
d = TextData()

## Calculate Accuracy

In [ ]:
correct_classifications = 0
pred_labels = text_model.predict([d.ids_test,
                        d.masks_test,
                        d.segments_test])

for actual_label, label in zip(d.labels_test, pred_labels):
      if actual_label.argmax(0) == label.argmax(0): # d.Classes[label.argmax(0)]: is the classname
            correct_classifications += 1

accuracy = float(correct_classifications) / d.test.shape[0] 
print(f'accuracy = {accuracy}')

accuracy = 0.846539883782356


# Calculate Accuracy for Fusion Model

## Load Model

In [ ]:
early_model = keras.models.load_model('early_fusion_weights_0.92.hdf5', custom_objects={'KerasLayer': hub.KerasLayer})

## Load Data

In [ ]:
batch_size = 300 # HINT: choose smaller batch_size if your machine runs out of memory
d = FusedData()
data_test = d.tf_data('images/test/*/*.jpg')
data_test = d.tf_data('images/test/*/*.jpg', batch_size=batch_size) 

## Calculate Accuracy

In [ ]:
correct_classifications = 0
i = 0
for ip, op in iter(data_test):
      batch_correct_classifications = 0
      images = ip['image']
      input_masks = ip['input_mask']
      input_word_ids = ip['input_word_ids']
      input_segments = ip['segment_ids']
      true_labels =  op['class']
      pred_labels = early_model.predict([input_word_ids,
                             input_masks,
                             input_segments,
                             images])

      for actual_label, label in zip(true_labels.numpy(), pred_labels):
            if actual_label.argmax(0) == label.argmax(0): # d.Classes[label.argmax(0)]: is the classname
                  correct_classifications += 1
                  batch_correct_classifications += 1
      batch_accuracy = float(batch_correct_classifications) / len(pred_labels) 
      i += 1
      print(f'accuracy for batch {i}/{num_test_samples / batch_size} = {batch_accuracy}')

accuracy = float(correct_classifications) / d.test.shape[0] 
print(f'accuracy = {accuracy}')

accuracy = 0.9249427716147209
